In [1]:
#import necessary libraries
import numpy as np
from scipy.sparse import csr_matrix
from scipy.sparse import lil_matrix
from pathlib import Path
import sys
sys.path.append("..")
from models.models import InnerProduct
import pandas as pd
import torch
import collections
import numpy as np
import torch.nn as nn
import os
import argparse
from data_processing.articles import Articles
from models.models import InnerProduct
import data_processing.dictionaries as dictionary
from pathlib import Path
import json
from pandas import json_normalize
from scipy import sparse
import boto3

dict_dir = Path("../../data/BERT/dictionaries")
final_word_ids,final_url_ids, final_publication_ids = dictionary.load_dictionaries(dict_dir)
print("Dictionaries loaded.")

with open('/users/rohan/news-classification/data/BERT/dictionaries/reversed_word_ids.json', "r") as file:
    id_to_word = json.load(file)

data_path = Path('../../data/final-data/train.json')
dataset = Articles(data_path)
print("Data loaded.")

dataset.tokenize()
proper_data = dataset.map_items(final_word_ids,
                    final_url_ids,
                    final_publication_ids,
                    filter=True,
                    min_length=200)

data_path = Path("/users/rohan/news-classification/Data/embedding-search/data")
if not data_path.is_dir():
    data_path.mkdir()
mapped_data_path = data_path / "mapped-data"
if not mapped_data_path.is_dir():
    mapped_data_path.mkdir()
train_mapped_path = mapped_data_path / "mapped_dataset.json"
with open(train_mapped_path, "w") as file:
    json.dump(proper_data, file)
raw_data = Articles(train_mapped_path)
print("Final: ", len(raw_data))
print(f"Filtered, Mapped Data saved to {mapped_data_path} directory")
print("-------------------")

In [17]:
abs_model_path = Path("../../data/gridsearch_results/2020-05-25/optimizer_type=RMS_use_all_words=False_emb_size=100_learning_rate=0.0001_word_embedding_type=mean/model/mean-inner-product-model.pt")
kwargs = dict(n_publications=len(final_publication_ids),
              n_articles=len(final_url_ids),
              n_attributes=len(final_word_ids),
              emb_size=100,
              sparse=False,
              use_article_emb=False,
              mode='mean')
model = InnerProduct(**kwargs)
model.load_state_dict(torch.load(abs_model_path))
print("Model Loaded.")

Model Loaded.


In [2]:
rows = []
cols = []
for idx, item in enumerate(raw_data.examples):
    word_ids = list(set(item['text']))
    number_of_words = np.arange(len(word_ids))
    rows.append(np.array(np.ones_like(number_of_words) * idx))
    cols.append(np.array(word_ids, dtype=np.int32))

In [3]:
final_rows = np.concatenate(rows, axis=None)
final_cols = np.concatenate(cols, axis=None)
final_data = np.ones_like(final_cols)

In [4]:
word_articles = csr_matrix((final_data, (final_rows, final_cols)))

In [18]:
print("Matrix Created!")

publication_emb = model.publication_embeddings.weight.data[0].cpu().numpy()
publication_bias = model.publication_bias.weight.data[0].cpu().numpy()
word_emb = model.attribute_emb_sum.weight.data.cpu().numpy()
word_bias = model.attribute_bias_sum.weight.data.cpu().numpy()

Matrix Created!


In [19]:
publication_emb

array([ 1.0440499 ,  1.0030843 ,  1.0340449 ,  0.992087  ,  1.0509816 ,
        1.0315005 , -1.0493797 , -1.0198538 ,  0.9712321 , -1.026394  ,
       -0.9687971 ,  1.0592866 , -1.0200703 , -1.0423145 ,  0.9929519 ,
        1.0220934 ,  1.021279  , -1.0265925 ,  0.9601833 ,  0.9763889 ,
        1.0109168 , -0.9728226 ,  0.97199583, -1.0237931 , -0.9996001 ,
        0.9932069 ,  0.97966635, -0.98893607, -0.9876815 , -0.98812914,
       -0.9625895 ,  0.99879754,  0.9876508 , -0.9581506 , -0.95436096,
       -0.9601925 , -1.0134513 , -0.98763955,  0.98665   , -1.0140482 ,
        1.004904  ,  0.9894275 , -1.0044671 , -0.9839679 , -0.97082543,
       -0.9798079 ,  0.9926766 , -0.97317344,  0.9797    , -0.97642475,
       -0.99420726, -0.9972062 , -1.0104703 ,  1.0575777 ,  0.9957696 ,
       -1.0413874 , -1.0056863 , -1.0151271 , -0.99969465,  0.97463423,
       -0.98398715, -1.0211866 , -1.0128828 , -1.0024365 , -0.9800189 ,
        1.0457181 ,  1.0155835 , -1.036794  , -1.013707  , -1.04

In [35]:
word_emb_indices = word_emb.argsort(axis=0)
print(word_emb_indices[:, 1][-100:])
print(word_emb_indices[:, 2][-100:])

[ 97058  74350   3422  13576  13336  17505   9678   6039   5244   8132
 103482   4337  10204   2768  59956  69910  62884   3486   5542  16999
   2667  31863  30074  46135 131804  19594   8315  18405  11226   4612
  12533  76975   5295   1064  83067   3398   9118 201038   2666  28494
 188036  20795  70641   1274   4016  73256   6429   4924   3400  11513
  16931   9608   3346  34346  10397  50922  18877  12284  15644  10817
    553  54456  11266   7501  61218  24594  94836   3393   2591  10464
  21082    251  21234  15394  17625  25837   3392   2602   4326  40904
  34602  61136   4243   3602  28584  18602   8292   8163   2352   1436
  13367  19263    903   9119  10078     26  32943   8640  16952    196]
[  1065  30817  33846  15394  67162  30163  61218  22415  61135 201038
  99425   1948 188036   3392   3251  16999  18877   5542   2666  15644
  18405  17832  21121  88959  28494   8063   4160   8315   1469   4016
   5295  73204  11226   3422  13576   8512  16931  50922  10817   4924
  137

In [ ]:
regularized_word_emb =  word_emb / np.sqrt(np.square(word_emb).sum(axis=-1)).reshape(448718, 1)
print(regularized_word_emb.shape)
word_emb_indices = regularized_word_emb.argsort(axis=0)
for idx in range(100):
    current_column = np.copy(word_emb_indices[:, idx][-150:])
    twisted_column = current_column.tolist()[::-1]
    top_words = []
    for index in twisted_column:
        top_words.append(id_to_word[str(index)])
    print(idx)
    if publication_emb[idx] > 0:
        print("Positive")
    else:
        print("Negative")
    print("---------------------")
    print(top_words)
    print("---------------------")

In [ ]:
article_embeddings = word_articles.dot(word_emb)
article_bias = word_articles.dot(word_bias)
word_counts = word_articles.sum(axis=1).reshape(word_articles.shape[0], 1)

for index in range(47, 100):
    altered_emb = np.copy(publication_emb)
    altered_emb[index] = 10000
    
    emb_times_publication = np.dot(article_embeddings, altered_emb.reshape(100,1))

    product_with_bias = emb_times_publication + article_bias

    final_logits = np.divide(product_with_bias, word_counts) 

    indices = final_logits.argsort(axis=0)[-75:].reshape(75)
    
    top_articles = word_articles[indices.tolist()[0]]
    
    word_logits = np.dot(word_emb, altered_emb.reshape(100,1)) + word_bias

    broadcasted_words_per_article = top_articles.toarray() * word_logits.T

    sorted_word_indices = broadcasted_words_per_article.argsort(axis=1)

    return_articles = []
    print(index)
    print("---------------------")
    return_articles = []
    i = 0
    for idx in indices.tolist()[0]:
        return_articles = []
        current_article = raw_data[int(idx)]
        current_article['logit'] = float(final_logits[int(idx)])
        current_sorted_words = sorted_word_indices[i]
        top_words = []
        least_words = []
        for top_word in current_sorted_words[-10:]:
            word = id_to_word[str(top_word)]
            top_words.append(word)
        for least_word in current_sorted_words[:10]:
            word = id_to_word[str(least_word)]
            least_words.append(word)
        current_article['top_words'] = top_words
        current_article['least_words'] = least_words
        return_articles.append(current_article)
        i += 1
        ordered_return_articles = return_articles[::-1]
        for article in ordered_return_articles:
            print(article['title'])
            print(article['link'])
            print(article['top_words'])
            print(article['least_words'])
            print(article['logit'])
        print("-----------------------------------------")

In [167]:
ranked = np.argsort(article_embeddings, axis=0)
ranked[1]

matrix([[ 13017,  13017,  13017,  32216,  13017,  13017, 110173, 111031,
          13017, 100152, 111889,  13017, 111031, 110173,  13017,  32216,
          32216, 110173,  13017,  13017,  13017, 111031,  13017, 102570,
         111031,  13017,  13017, 109602, 100152, 111806, 100152,  13017,
          32216, 100152, 100152, 102570, 110173, 100152,  13017, 111031,
          13017,  13017, 111031, 102570, 100152, 111806,  13017, 109602,
          13017, 110173, 111031, 110173, 100152,  13017,  13017, 110173,
         109602, 102570, 111031,  32216, 109602, 109602, 102570, 106597,
         100152,  13017,  13017, 100152, 111031, 109602, 109602, 110173,
         109602,  13017,  13017,  13017,  32216, 106597,  13017, 106597,
         100152, 110173, 102570, 109602,  13017,  13017,  13017,  13017,
          13017,  13017,  13017,  13017,  13017,  13017, 100152, 107710,
         100152,  13017,  13017, 111031]], dtype=int64)

In [ ]:
for idx in range(100):
    current_iteration = ranked[:, idx][::-1]
    listed_iteration = np.squeeze(current_iteration).tolist()[0]
    print(idx)
    print("-----------------------------")
    for top in range(20):   
        print(listed_iteration[top], raw_data[listed_iteration[top]]['title'])
        print(article_embeddings[listed_iteration[top], idx])
    print("-----------------------------")

In [4]:
#import necessary libraries
import numpy as np
from scipy.sparse import csr_matrix
from scipy.sparse import lil_matrix
from pathlib import Path
import sys
sys.path.append("../bert-approach")
import pandas as pd
import torch
import collections
import numpy as np
import torch.nn as nn
import os
import argparse
from data_processing.articles import Articles
from models.models import InnerProduct
import data_processing.dictionaries as dictionary
from pathlib import Path
import json
from pandas import json_normalize
from scipy import sparse
import boto3

dict_dir = Path("../../data/BERT/dictionaries")
final_word_ids,final_url_ids, final_publication_ids = dictionary.load_dictionaries(dict_dir)

In [7]:
reversed_word_ids = {v: k for k, v in final_word_ids.items()}
with open('/users/rohan/news-classification/data/BERT/dictionaries/reversed_word_ids.json', "w") as file:
    json.dump(reversed_word_ids, file)